In [1]:
import pandas as pd
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sentence_transformers import SentenceTransformer
from sklearn.ensemble import RandomForestClassifier

In [2]:
df = pd.read_csv("../../data/final_data.csv")
df.head()

,id,job_title,location,salary_currency,career_level,experience_level,education_level,employment_type,job_function,job_benefits,...,company_industry,job_description,salary,job_description_cleaned,annotations,skills,career_level_norm,experience_years,experience_bucket,education_level_norm
0,10,Tax Supervisor,Banten,IDR,Supervisor/Koordinator,3 tahun,"Sertifikat Professional, D3 (Diploma), D4 (Dip...",Penuh Waktu,"Akuntansi / Keuangan,Audit & Pajak","Tip;Asuransi kesehatan;Waktu regular, Senin - ...",...,Konstruksi/Bangunan/Teknik,ResponsibilitiesPrepare corporate income tax (...,8000000.0,responsibilitiesprepare corporate income tax c...,{'text': 'responsibilitiesprepare corporate in...,"['income tax', 'tax compliance', 'digital elec...",Supervisor,3,Middle,S1
1,27,MARKETING COMMUNICATION & CHANNEL MANAGER,Jakarta Pusat,IDR,Manajer/Asisten Manajer,2 tahun,Tidak terspesifikasi,Penuh Waktu,"Penjualan / Pemasaran,Pemasaran/Pengembangan B...",Asuransi Gigi;Asuransi kesehatan;Parkir;Waktu ...,...,"Konsultasi (IT, Ilmu Pengetahuan, Teknis & Tek...","SCOPE OF ROLE :To plan, develop, implement and...",15000000.0,scope of role to plan develop implement and me...,{'text': 'scope of role to plan develop implem...,"['marketing communication', 'strategic communi...",Manajer,2,Middle,Unknown
2,32,PPIC,Tangerang,IDR,Manajer/Asisten Manajer,5 tahun,Sarjana (S1),Penuh Waktu,"Manufaktur,Manufaktur","Asuransi kesehatan;Waktu regular, Senin - Juma...",...,Manufaktur/Produksi,Tanggung Jawab:Membuat perencanaan produksi be...,6600000.0,tanggung jawab membuat perencanaan produksi be...,{'text': 'tanggung jawab membuat perencanaan p...,"['warehouse management', 'inventory control', ...",Manajer,5,Senior,S1
3,35,C# Advanced/Senior Developer,Jakarta Timur,IDR,Supervisor/Koordinator,2 tahun,"SMA, SMU/SMK/STM, Sertifikat Professional, D3 ...",Penuh Waktu,"Komputer/Teknologi Informasi,IT-Perangkat Lunak","Tip;Asuransi kesehatan;Waktu regular, Senin - ...",...,Komputer/Teknik Informatika (Perangkat Lunak),Kualifikasi:Kandidat harus memiliki setidaknya...,9500000.0,kualifikasi kandidat harus memiliki setidaknya...,{'text': 'kualifikasi kandidat harus memiliki ...,"['asp net', 'object orient programming', 'c', ...",Supervisor,2,Middle,S1
4,38,IT Software Developer Staff,Jakarta Barat,IDR,Pegawai (non-manajemen & non-supervisor),1 tahun,"Sarjana (S1), Diploma Pascasarjana, Gelar Prof...",Penuh Waktu,"Komputer/Teknologi Informasi,IT-Perangkat Lunak",Kasual (contoh: Kaos);Senin - Jum'at (HO) Seni...,...,Industri Berat/Mesin/Peralatan,Bachelor’s degree of Information Technology or...,6800000.0,bachelor’s degree of information technology or...,{'text': 'bachelor’s degree of information tec...,"['information technology', 'restful api', 'spr...",Staff,1,Entry,S2


In [3]:
# Get a NumPy array of all unique values
unique_values_list = df['company_industry'].unique()
print(unique_values_list)

['Konstruksi/Bangunan/Teknik'
 'Konsultasi (IT, Ilmu Pengetahuan, Teknis & Teknikal)'
 'Manufaktur/Produksi' 'Komputer/Teknik Informatika (Perangkat Lunak)'
 'Industri Berat/Mesin/Peralatan' 'Makanan & Minuman/Katering/Restoran'
 'Periklanan/Marketing/Promosi/Hubungan Masyarakat'
 'Perbankan/Pelayanan Keuangan' 'Telekomunikasi' 'Kayu/Fiber/Kertas'
 'Properti/Real Estate' 'Umum & Grosir' 'Retail/Merchandise'
 'Produk Konsumen/Barang konsumen yang bergerak cepat' 'Pertambangan'
 'Asuransi' 'Automobil/Mesin Tambahan Automotif/Kendaraan'
 'Manajemen/Konsulting HR' 'Akunting / Audit / Layanan Pajak'
 'Komputer/Teknik Informatika (Perangkat Keras)' 'Transportasi/Logistik'
 'Pendidikan' 'Kesehatan/Medis' 'Konsultasi (Bisnis & Manajemen)'
 'Bahan Kimia/Pupuk/Pestisida' 'Elektrikal & Elektronik'
 'Pelayanan Perbaikan & Pemeliharaan' 'Perawatan/Kecantikan/Fitnes'
 'Pelayanan Arsitek/Desain Interior' 'Lingkungan/Kesehatan/Keamanan'
 'Layanan Umum/Tenaga Penggerak' 'Percetakan/Penerbitan' 'Tekstil

In [4]:
industry_map = {
    # Teknologi & IT
    "Komputer/Teknik Informatika (Perangkat Lunak)": "Teknologi & IT",
    "Komputer/Teknik Informatika (Perangkat Keras)": "Teknologi & IT",
    "Telekomunikasi": "Teknologi & IT",
    "Ilmu Pengetahuan & Teknologi": "Teknologi & IT",
    "Call Center/IT-Enabled Services/BPO": "Teknologi & IT",

    # Manufaktur & Industri
    "Manufaktur/Produksi": "Manufaktur & Industri",
    "Industri Berat/Mesin/Peralatan": "Manufaktur & Industri",
    "Elektrikal & Elektronik": "Manufaktur & Industri",
    "Automobil/Mesin Tambahan Automotif/Kendaraan": "Manufaktur & Industri",
    "Polymer/Plastik/Karet/Ban": "Manufaktur & Industri",
    "Tekstil/Garment": "Manufaktur & Industri",
    "Kayu/Fiber/Kertas": "Manufaktur & Industri",
    "Percetakan/Penerbitan": "Manufaktur & Industri",
    "Bahan Kimia/Pupuk/Pestisida": "Manufaktur & Industri",

    # Konstruksi & Properti
    "Konstruksi/Bangunan/Teknik": "Konstruksi & Properti",
    "Properti/Real Estate": "Konstruksi & Properti",
    "Pelayanan Arsitek/Desain Interior": "Konstruksi & Properti",

    # Keuangan & Asuransi
    "Perbankan/Pelayanan Keuangan": "Keuangan & Asuransi",
    "Asuransi": "Keuangan & Asuransi",
    "Jual Beli Saham/Sekuritas": "Keuangan & Asuransi",
    "Akunting / Audit / Layanan Pajak": "Keuangan & Asuransi",

    # Kesehatan & Life Sciences
    "Kesehatan/Medis": "Kesehatan & Life Sciences",
    "BioTeknologi/Farmasi/Riset klinik": "Kesehatan & Life Sciences",
    "Lingkungan/Kesehatan/Keamanan": "Kesehatan & Life Sciences",

    # Pendidikan & Riset
    "Pendidikan": "Pendidikan & Riset",

    # Energi & Pertambangan
    "Pertambangan": "Energi & Pertambangan",
    "Minyak/Gas/Petroleum": "Energi & Pertambangan",
    "Layanan Umum/Tenaga Penggerak": "Energi & Pertambangan",

    # Retail, FMCG & Konsumen
    "Retail/Merchandise": "Retail, FMCG & Konsumen",
    "Umum & Grosir": "Retail, FMCG & Konsumen",
    "Produk Konsumen/Barang konsumen yang bergerak cepat": "Retail, FMCG & Konsumen",
    "Makanan & Minuman/Katering/Restoran": "Retail, FMCG & Konsumen",
    "Pakaian": "Retail, FMCG & Konsumen",
    "Tembakau": "Retail, FMCG & Konsumen",
    "Perawatan/Kecantikan/Fitnes": "Retail, FMCG & Konsumen",
    "Permata/Perhiasan": "Retail, FMCG & Konsumen",

    # Transportasi & Logistik
    "Transportasi/Logistik": "Transportasi & Logistik",
    "Luar Angkasa/Aviasi/Pesawat Terbang": "Transportasi & Logistik",

    # Media, Kreatif & Hiburan
    "Hiburan/Media": "Media, Kreatif & Hiburan",
    "Jurnalisme": "Media, Kreatif & Hiburan",
    "Seni/Desain/Fashion": "Media, Kreatif & Hiburan",
    "Periklanan/Marketing/Promosi/Hubungan Masyarakat": "Media, Kreatif & Hiburan",
    "Pameran/Manajemen acara/PIKP": "Media, Kreatif & Hiburan",

    # Jasa Profesional & Konsultasi
    "Konsultasi (IT, Ilmu Pengetahuan, Teknis & Teknikal)": "Jasa Profesional & Konsultasi",
    "Konsultasi (Bisnis & Manajemen)": "Jasa Profesional & Konsultasi",
    "Manajemen/Konsulting HR": "Jasa Profesional & Konsultasi",
    "Hukum/Legal": "Jasa Profesional & Konsultasi",
    "Pelayanan Perbaikan & Pemeliharaan": "Jasa Profesional & Konsultasi",

    # Hospitality & Pariwisata
    "Hotel/Pariwisata": "Hospitality & Pariwisata",
    "Travel/Pariwisata": "Hospitality & Pariwisata",

    # Pertanian & Kelautan
    "Agrikultural/Perkebunan/Peternakan Unggas/Perikanan": "Pertanian & Kelautan",
    "Kelautan/Aquakultur": "Pertanian & Kelautan",

    # Publik, Sosial & Keamanan
    "Organisasi Nirlaba/Pelayanan Sosial/LSM": "Publik, Sosial & Keamanan",
    "Keamanan/Penegak hukum": "Publik, Sosial & Keamanan",

    # Lainnya
    "Lainnya": "Lainnya"
}

df["company_industry_simple"] = df["company_industry"].map(industry_map)

In [5]:
encoders_dict = {}

columns_to_encode = ['employment_type', 'career_level_norm', 'experience_years', 'experience_bucket', 'education_level_norm',
                    'job_function', 'company_industry_simple']
for col in columns_to_encode:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    encoders_dict[col] = le

df.head()

,id,job_title,location,salary_currency,career_level,experience_level,education_level,employment_type,job_function,job_benefits,...,job_description,salary,job_description_cleaned,annotations,skills,career_level_norm,experience_years,experience_bucket,education_level_norm,company_industry_simple
0,10,Tax Supervisor,Banten,IDR,Supervisor/Koordinator,3 tahun,"Sertifikat Professional, D3 (Diploma), D4 (Dip...",3,1,"Tip;Asuransi kesehatan;Waktu regular, Senin - ...",...,ResponsibilitiesPrepare corporate income tax (...,8000000.0,responsibilitiesprepare corporate income tax c...,{'text': 'responsibilitiesprepare corporate in...,"['income tax', 'tax compliance', 'digital elec...",4,2,1,1,5
1,27,MARKETING COMMUNICATION & CHANNEL MANAGER,Jakarta Pusat,IDR,Manajer/Asisten Manajer,2 tahun,Tidak terspesifikasi,3,34,Asuransi Gigi;Asuransi kesehatan;Parkir;Waktu ...,...,"SCOPE OF ROLE :To plan, develop, implement and...",15000000.0,scope of role to plan develop implement and me...,{'text': 'scope of role to plan develop implem...,"['marketing communication', 'strategic communi...",2,1,1,5,2
2,32,PPIC,Tangerang,IDR,Manajer/Asisten Manajer,5 tahun,Sarjana (S1),3,19,"Asuransi kesehatan;Waktu regular, Senin - Juma...",...,Tanggung Jawab:Membuat perencanaan produksi be...,6600000.0,tanggung jawab membuat perencanaan produksi be...,{'text': 'tanggung jawab membuat perencanaan p...,"['warehouse management', 'inventory control', ...",2,4,2,1,7
3,35,C# Advanced/Senior Developer,Jakarta Timur,IDR,Supervisor/Koordinator,2 tahun,"SMA, SMU/SMK/STM, Sertifikat Professional, D3 ...",3,12,"Tip;Asuransi kesehatan;Waktu regular, Senin - ...",...,Kualifikasi:Kandidat harus memiliki setidaknya...,9500000.0,kualifikasi kandidat harus memiliki setidaknya...,{'text': 'kualifikasi kandidat harus memiliki ...,"['asp net', 'object orient programming', 'c', ...",4,1,1,1,13
4,38,IT Software Developer Staff,Jakarta Barat,IDR,Pegawai (non-manajemen & non-supervisor),1 tahun,"Sarjana (S1), Diploma Pascasarjana, Gelar Prof...",3,12,Kasual (contoh: Kaos);Senin - Jum'at (HO) Seni...,...,Bachelor’s degree of Information Technology or...,6800000.0,bachelor’s degree of information technology or...,{'text': 'bachelor’s degree of information tec...,"['information technology', 'restful api', 'spr...",3,0,0,2,7


In [63]:
df["role_text"] = df["job_title"] + " - " + df['skills'] ## + " - " + df["job_description_cleaned"]

In [7]:
EMBED_MODEL_NAME = "paraphrase-multilingual-MiniLM-L12-v2"
embed_model = SentenceTransformer(EMBED_MODEL_NAME)

def embed_texts(texts, batch_size=64):
    return embed_model.encode(texts, show_progress_bar=True, batch_size=batch_size, convert_to_numpy=True)

role_embeddings = embed_texts(df["role_text"].astype(str).tolist())

df["role_embedding"] = list(role_embeddings)

Batches:   0%|          | 0/46 [00:00<?, ?it/s]

In [8]:
df.head()

,id,job_title,location,salary_currency,career_level,experience_level,education_level,employment_type,job_function,job_benefits,...,job_description_cleaned,annotations,skills,career_level_norm,experience_years,experience_bucket,education_level_norm,company_industry_simple,role_text,role_embedding
0,10,Tax Supervisor,Banten,IDR,Supervisor/Koordinator,3 tahun,"Sertifikat Professional, D3 (Diploma), D4 (Dip...",3,1,"Tip;Asuransi kesehatan;Waktu regular, Senin - ...",...,responsibilitiesprepare corporate income tax c...,{'text': 'responsibilitiesprepare corporate in...,"['income tax', 'tax compliance', 'digital elec...",4,2,1,1,5,"Tax Supervisor - ['income tax', 'tax complianc...","[0.17798801, 0.25791928, -0.10339857, 0.078521..."
1,27,MARKETING COMMUNICATION & CHANNEL MANAGER,Jakarta Pusat,IDR,Manajer/Asisten Manajer,2 tahun,Tidak terspesifikasi,3,34,Asuransi Gigi;Asuransi kesehatan;Parkir;Waktu ...,...,scope of role to plan develop implement and me...,{'text': 'scope of role to plan develop implem...,"['marketing communication', 'strategic communi...",2,1,1,5,2,MARKETING COMMUNICATION & CHANNEL MANAGER - ['...,"[0.2701247, 0.10016379, 0.08252446, -0.2087123..."
2,32,PPIC,Tangerang,IDR,Manajer/Asisten Manajer,5 tahun,Sarjana (S1),3,19,"Asuransi kesehatan;Waktu regular, Senin - Juma...",...,tanggung jawab membuat perencanaan produksi be...,{'text': 'tanggung jawab membuat perencanaan p...,"['warehouse management', 'inventory control', ...",2,4,2,1,7,"PPIC - ['warehouse management', 'inventory con...","[-0.042296365, -0.11403816, 0.09667922, 0.0634..."
3,35,C# Advanced/Senior Developer,Jakarta Timur,IDR,Supervisor/Koordinator,2 tahun,"SMA, SMU/SMK/STM, Sertifikat Professional, D3 ...",3,12,"Tip;Asuransi kesehatan;Waktu regular, Senin - ...",...,kualifikasi kandidat harus memiliki setidaknya...,{'text': 'kualifikasi kandidat harus memiliki ...,"['asp net', 'object orient programming', 'c', ...",4,1,1,1,13,"C# Advanced/Senior Developer - ['asp net', 'ob...","[0.011605643, -0.028905015, 0.05907163, 0.0966..."
4,38,IT Software Developer Staff,Jakarta Barat,IDR,Pegawai (non-manajemen & non-supervisor),1 tahun,"Sarjana (S1), Diploma Pascasarjana, Gelar Prof...",3,12,Kasual (contoh: Kaos);Senin - Jum'at (HO) Seni...,...,bachelor’s degree of information technology or...,{'text': 'bachelor’s degree of information tec...,"['information technology', 'restful api', 'spr...",3,0,0,2,7,IT Software Developer Staff - ['information te...,"[-0.15960664, -0.20797366, -0.022929283, -0.12..."


In [9]:
X_emb = np.hstack([role_embeddings])
print("X_emb shape:", X_emb.shape)

num_features = ['employment_type', 'career_level_norm', 'experience_years', 'experience_bucket', 'education_level_norm',
                    'job_function']

if len(num_features) > 0:
    cat_values = df[num_features].values.astype(np.float32)
    X = np.hstack([X_emb, cat_values])
else:
    X = X_emb

print("Final feature X shape:", X.shape)

y = df["company_industry_simple"].fillna("unknown")
print("n_classes:", y.nunique())
print(y.value_counts().head())

X_emb shape: (2921, 384)
Final feature X shape: (2921, 390)
n_classes: 15
company_industry_simple
12    594
7     482
13    453
2     443
4     256
Name: count, dtype: int64


In [10]:
X_train, X_test, y_train, y_test = train_test_split(X_emb, y, test_size=0.3, random_state=42)

In [11]:
# Train the Random Forest model
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

# Evaluate the model
accuracy_before = rf.score(X_test, y_test)
print(f'Accuracy before feature selection: {accuracy_before:.2f}')

Accuracy before feature selection: 0.53


In [12]:
embedding_dim = X_emb.shape[1]
feature_names = (
    [f"role_embedding_{i}" for i in range(embedding_dim)]
    # + num_features
)

feature_importance_df = pd.DataFrame({
    "Feature": feature_names,
    "Importance": rf.feature_importances_
}).sort_values("Importance", ascending=False)
print(feature_importance_df)

# Select top N features (example selecting top 10 features)
top_feature_indices = feature_importance_df.index[:10].values
X_train_selected = X_train[:, top_feature_indices]
X_test_selected = X_test[:, top_feature_indices]

                Feature  Importance
62    role_embedding_62    0.005103
271  role_embedding_271    0.005042
157  role_embedding_157    0.004902
247  role_embedding_247    0.004871
138  role_embedding_138    0.004783
..                  ...         ...
313  role_embedding_313    0.001845
339  role_embedding_339    0.001841
228  role_embedding_228    0.001821
277  role_embedding_277    0.001794
312  role_embedding_312    0.001742

[384 rows x 2 columns]


In [13]:
# Train the Random Forest model with selected features
rf_selected = RandomForestClassifier(n_estimators=100, random_state=42)
rf_selected.fit(X_train_selected, y_train)

# Evaluate the model
accuracy_after = rf_selected.score(X_test_selected, y_test)
print(f'Accuracy after feature selection: {accuracy_after:.2f}')

Accuracy after feature selection: 0.46


## TFIDF

In [64]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [65]:
tfidf = TfidfVectorizer(
    max_features=50,        # penting agar RF tidak overload
    ngram_range=(1, 2),
    # stop_words="english",
    min_df=2,
    max_df=0.9,
    sublinear_tf=True
)

In [66]:
X_tfidf = tfidf.fit_transform(df["role_text"].astype(str))

In [67]:
X_tfidf_dense = X_tfidf.toarray()
X_emb = X_tfidf_dense
print("TF-IDF shape:", X_emb.shape)

TF-IDF shape: (2921, 50)


In [68]:
if len(num_features) > 0:
    cat_values = df[num_features].values.astype(np.float32)
    X = np.hstack([X_emb, cat_values])
else:
    X = X_emb

print("Final X shape:", X.shape)

Final X shape: (2921, 56)


In [69]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [70]:
# Train the Random Forest model
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

# Evaluate the model
accuracy_before = rf.score(X_test, y_test)
print(f'Accuracy before feature selection: {accuracy_before:.2f}')

Accuracy before feature selection: 0.52


In [71]:
tfidf_feature_names = tfidf.get_feature_names_out()

feature_names = list(tfidf_feature_names) + num_features
feature_importance_df = pd.DataFrame({
    "Feature": feature_names,
    "Importance": rf.feature_importances_
}).sort_values("Importance", ascending=False)

In [72]:
print(feature_importance_df)

top_feature_indices = feature_importance_df.index[:10].values

X_train_selected = X_train[:, top_feature_indices]
X_test_selected = X_test[:, top_feature_indices]

                 Feature  Importance
55          job_function    0.104001
39                 sales    0.043518
52      experience_years    0.040500
22            management    0.037115
5          communication    0.035884
54  education_level_norm    0.034971
16               english    0.033367
51     career_level_norm    0.029676
12           development    0.027177
23               manager    0.026999
17             executive    0.024516
53     experience_bucket    0.023915
25             marketing    0.021412
4               business    0.020307
2                 bahasa    0.018353
50       employment_type    0.016950
27             microsoft    0.016917
40               service    0.016462
3                  brand    0.016042
47                system    0.015445
20            leadership    0.015375
45              software    0.014905
11                design    0.014676
31                  plan    0.014461
0             accounting    0.014336
1             analytical    0.014193
2

In [73]:
# Train the Random Forest model with selected features
rf_selected = RandomForestClassifier(n_estimators=100, random_state=42)
rf_selected.fit(X_train_selected, y_train)

# Evaluate the model
accuracy_after = rf_selected.score(X_test_selected, y_test)
print(f'Accuracy after feature selection: {accuracy_after:.2f}')

Accuracy after feature selection: 0.49


## Logistic Regression

In [74]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(
    max_iter=3000,
    n_jobs=-1,
    class_weight="balanced",
    solver="lbfgs"
)

clf.fit(X_train, y_train)

LogisticRegression(class_weight='balanced', max_iter=3000, n_jobs=-1)

In [75]:
from sklearn.metrics import accuracy_score, classification_report

y_pred = clf.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Accuracy: 0.24059293044469784
              precision    recall  f1-score   support

           0       0.07      0.45      0.12        11
           1       0.02      0.50      0.05         2
           2       0.51      0.17      0.26       132
           3       0.17      0.21      0.19        34
           4       0.51      0.57      0.54        89
           5       0.24      0.27      0.25        56
           6       0.04      0.20      0.06        10
           7       0.40      0.11      0.17       145
           8       0.05      0.19      0.08        16
           9       0.16      0.43      0.23        21
          10       0.04      0.43      0.07         7
          11       0.03      0.33      0.06         3
          12       0.45      0.10      0.16       177
          13       0.54      0.33      0.41       150
          14       0.13      0.33      0.18        24

    accuracy                           0.24       877
   macro avg       0.22      0.31      0.19       

In [76]:
tfidf_features = tfidf.get_feature_names_out()
all_features = list(tfidf_features) + num_features

In [77]:
import numpy as np

class_idx = list(clf.classes_).index(5)  # ganti sesuai kelas

coef = clf.coef_[class_idx]
top_idx = np.argsort(coef)[-20:]

for i in reversed(top_idx):
    print(all_features[i], coef[i])

project 1.5883969135640807
professional 1.5258774880255883
engineer 1.3595228205947456
engineering 1.0261582632987012
control 0.8939655508717992
design 0.7411320970571411
office 0.6891055273860324
marketing 0.6658950675816362
digital 0.572657560706936
plans 0.48913555740126424
bahasa 0.3018554521979711
business 0.29327907562866107
career_level_norm 0.2651441517372056
brand 0.25643823472537797
microsoft 0.21015830227273027
medium 0.15460130353415602
english 0.13584660351942307
experience_bucket 0.10729117437778211
process 0.086949526998704
manage 0.08048620835819356


In [78]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2921 entries, 0 to 2920
Data columns (total 25 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       2921 non-null   int64  
 1   job_title                2921 non-null   object 
 2   location                 2921 non-null   object 
 3   salary_currency          2921 non-null   object 
 4   career_level             2921 non-null   object 
 5   experience_level         2921 non-null   object 
 6   education_level          2921 non-null   object 
 7   employment_type          2921 non-null   int64  
 8   job_function             2921 non-null   int64  
 9   job_benefits             2921 non-null   object 
 10  company_process_time     2921 non-null   object 
 11  company_size             2921 non-null   object 
 12  company_industry         2921 non-null   object 
 13  job_description          2921 non-null   object 
 14  salary                  